In [1]:
import requests
import re

In [2]:
class stationDataHandler():
    def __init__(self):
        self.baseUrl='http://www.cehq.gouv.qc.ca/suivihydro/'
    def getStations(self):
        """Fetches all station IDs
        """
        stationRegex = re.compile('.*option value="[0-9]')
        self.stations=[]
        try:
            source=requests.get('%sdefault.asp' % (self.baseUrl))
            for line in source.iter_lines():
                if line.isascii() and re.match(stationRegex, line.decode('utf-8')):
                    station=line.decode('utf-8').split('"')[1].strip()
                    #This is just to avoid dupes
                    if station not in self.stations:
                        self.stations.append(station)
                    else:
                        print('Duplicate station ID found: {}'.format(station))
        except:
            print("Unexpected error:", sys.exc_info()[0])
            raise
    def stationMetaFetch(self, station):
        """Function to gather the details of the station
        
        station - int - ID of the station
        """
        #TODO: everything
        print('should do this')

    def stationDataFetch(self, station):
        source=requests.get('{}fichier_donnees.asp?NoStation={}'.format(self.baseUrl, station))
        stationData=[]
        for line in source.iter_lines():
            if line.startswith(b'D'):
                columns=line.replace(b'\xe9', b'e').decode('utf-8').split()
            elif not line.startswith(b'D') and not line == b'' and not line.startswith(b'A'):
                #nLine=line.replace(b'\xa0', b'').decode('utf-8').replace('\t', ' ')
                #stationData.append('{}\n'.format(nLine.replace(',', '.')))
                nLine=line.replace(b'\xa0', b'').decode('utf-8').replace(',', '.').split()
                stationData.append(' '.join(map(lambda x: '[{}]{}'.format(x[0], x[1]), zip(columns, nLine))))
        return stationData
    def writeStationData(self, station):
        data=self.stationDataFetch(station)
        if station not in self.stations:
            #TODO: should instead try to fetch it, if not real then error
            raise Exception('station id has not been discovered, likely wrong number')
        
        with open('./data/{}'.format(station), 'w') as file:
            for line in data:
                file.write('{}\n'.format(line))

In [ ]:
o=stationDataHandler()
o.getStations()
#print(o.stations)
for station in o.stations:
    o.writeStationData(station)
#o.writeStationData('043301')

In [20]:
import platform
import socket
import time
import datetime

data = requests.get('http://www.cehq.gouv.qc.ca/suivihydro/fichier_donnees.asp?NoStation=043301')


CARBON_SERVER = '0.0.0.0'
CARBON_PORT = 2003
DELAY = 10

def send_msg(message):
    print('Sending message:\n%s' % message)
    sock = socket.socket()
    sock.connect((CARBON_SERVER, CARBON_PORT))
    sock.sendall(bytearray(message, 'utf-8'))
    sock.close()

In [21]:
for line in data.iter_lines():
    if line.startswith(b'D'):
        columns = line.replace(b'\xe9', b'e').decode('utf-8').split()
    elif not line == b'':
        nLine = line.replace(b'\xa0', b'').decode('utf-8').replace(',', '.').split()
        year, month, day = map(lambda x: int(x), nLine[0].split('-'))
        hour, minute = map(lambda x: int(x), nLine[1].split(':'))
        timestamp = int(datetime.datetime(year, month, day, hour, minute).timestamp())
        #send_msg(bytearray(' '.join(map(lambda x: '[{}]{}'.format(x[0], x[1]), zip(columns, nLine))), 'utf-8'))
        #print(' '.join(map(lambda x: '[{}]{}'.format(x[0], x[1]), zip(columns, nLine))))
        for metric in range(2,len(columns)):
            send_msg('station.043301.{} {} {}'.format(columns[metric], nLine[metric], timestamp))
            time.sleep(DELAY)

Sending message:
station.043301.Niveau 22.16 1557322200
Sending message:
station.043301.Debit 2949 1557322200
Sending message:
station.043301.Niveau 22.17 1557321300
Sending message:
station.043301.Debit 2958 1557321300
Sending message:
station.043301.Niveau 22.15 1557320400
Sending message:
station.043301.Debit 2943 1557320400
Sending message:
station.043301.Niveau 22.17 1557319500
Sending message:
station.043301.Debit 2953 1557319500
Sending message:
station.043301.Niveau 22.18 1557318600
Sending message:
station.043301.Debit 2966 1557318600
Sending message:
station.043301.Niveau 22.17 1557317700
Sending message:
station.043301.Debit 2953 1557317700
Sending message:
station.043301.Niveau 22.17 1557316800
Sending message:
station.043301.Debit 2954 1557316800
Sending message:
station.043301.Niveau 22.16 1557315900
Sending message:
station.043301.Debit 2946 1557315900
Sending message:
station.043301.Niveau 22.18 1557315000
Sending message:
station.043301.Debit 2960 1557315000
Sending me

ConnectionRefusedError: [Errno 111] Connection refused

In [15]:
#print(columns)
#print({columns[i]:nLine[i].replace(',', '.') for i in range(len(columns))})
#print([nLine[i].replace(',', '.') for i in range(len(columns))])

#' '.join(map(lambda x: '[{}]{}'.format(x[0], x[1]), zip(columns, nLine)))
test = ['2019-05-07', '14:45', '22.20', '2982']
year, month, day = map(lambda x: int(x), test[0].split('-'))
hour, minute = map(lambda x: int(x), test[1].split(':'))
epoch = int(datetime.datetime(year, month, day, hour, minute).timestamp())
print(epoch)

1557254700
